In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from sklearn.model_selection import train_test_split
import pathlib
from sklearn.utils import shuffle

In [ ]:
dataframe = pd.read_csv("../input/breast-cancer-prediction/Breast Cancer Prediction.csv")
dataframe = dataframe.drop(columns="Sample code number")
dataframe = dataframe.rename(columns={
    "Clump Thickness": "ClumpThickness",
    "Uniformity of Cell Size": "UniformityOfCellSize",
    "Uniformity of Cell Shape": "UniformityOfCellShape",
    "Marginal Adhesion": "MarginalAdhesion",
    "Single Epithelial Cell Size": "SingleEpithelialCellSize",
    "Bare Nuclei": "BareNuclei",
    "Bland Chromatin": "BlandChromatin",
    "Normal Nucleoli": "NormalNucleoli",
    "Class": "Malignant"
})
dataframe = shuffle(dataframe)
dataframe.loc[dataframe["Malignant"]==2, "Malignant"] = 0
dataframe.loc[dataframe["Malignant"]==4, "Malignant"] = 1
dataframe = dataframe.astype(float)

In [ ]:
allColumns = list(dataframe.columns)
allColumns.remove("Malignant")

In [ ]:
train, val = train_test_split(dataframe, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'val examples')

In [ ]:
minMaxScalar = {}

for column in allColumns:
    high = train[column].max()
    low = train[column].min()
    minMaxScalar[column] = {"low": low, "high": high}
    train[column] = (train[column] - low) / (high - low)
    val[column] = (val[column] - low) / (high - low)

In [ ]:
def df_to_dataset(df, shuffle=True, batch_size=128):
    df = df.copy()
    labels = df.pop("Malignant")
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [ ]:
feature_columns = []

for column in allColumns:
    feature_columns.append(feature_column.numeric_column(column))

In [ ]:
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val, shuffle=False)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
history = model.fit(train_ds, validation_data=val_ds, epochs=100, verbose=2)

In [ ]:
accuracy=history.history['accuracy']
val_accuracy=history.history['val_accuracy']

epocRanges=range(len(accuracy))
plt.plot(epocRanges, accuracy)
plt.plot(epocRanges, val_accuracy)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["accuracy","val_accuracy"])
plt.figure()